# Experimentos

Experimentos são a principal forma de agrupar 3 entidades:

1. **Datasets**: conjunto de textos acompanhados de um *target*, representando um problema de classificação ou regressão;
2. **Pipelines**: são capazes de receber um ou mais textos como entrada e produzir uma saída para o problema de classificação/regressão;
3. **Métricas**: permitem avaliar o desempenho de uma ou mais pipelines para o problema em questão;

A biblioteca `aibox-nlp` disponibiliza diferentes métodos para construção e execução de experimentos em seu pacote `aibox.nlp.experiments`. É possível construir um experimento instanciando cada um dos componentes individualmente ou através do pacote `aibox.nlp.factory`, que possui facilidades para obter as diversas classes da biblioteca.

In [1]:
import json

import pandas as pd

from aibox.nlp import serialization
from aibox.nlp.factory.experiment import SimpleExperimentBuilder

## Exemplo 1 - Construindo um Experimento

In [2]:
# === Construindo um experimento para classificação no Essay-BR estendido ===
# Por simplicidade, vamos instanciar um experimento
#   para comparar algumas abordagens para classificação
#   da competência 1 do dataset Essay-BR.
builder = SimpleExperimentBuilder()

# Inicialmente, vamos definir o dataset
builder.dataset('essayBR',
                extended=True,
                target_competence='C1')

# Também é possível passar uma instância
#   de um Dataset diretamente:
# builder.custom_dataset(ds)

# Vamos definir o tipo do problema
builder.classification()

# Vamos definir a seed randômica
builder.seed(42)

# Depois, vamos definir algumas métricas
#   que devem ser calculadas
builder.add_metric('precision', average='weighted')
builder.add_metric('recall', average='weighted')
builder.add_metric('f1', average='weighted')
builder.add_metric('kappa')
builder.add_metric('neighborKappa')

# Depois, vamos definir qual a métrica
#   que deve ser utilizar para escolher a
#   melhor pipeline
builder.best_criteria('precision', average='weighted')

# Agora, vamos adicionar algumas pipelines baseadas
#   em extração de característica
builder.add_feature_pipeline(
    features=['readabilityBR',
              'regencyBR',
              'syntacticComplexityBR',
              'textualSimplicityBR'],
    estimators=['svm',
                'etreesClf',
                'lgbmClf',
                'xgbClf'],
    names=['svm+features',
           'etrees+features',
           'lgbm+features',
           'xgb+features'])

# Agora, vamos adicionar algumas pipelines baseadas
#   em outras estratégias de vetorização
builder.add_vectorizer_pipeline('tfidfVectorizer',
                                estimators=['etreesClf',
                                            'lgbmClf',
                                            'xgbClf'],
                                names=['etrees+tfidf',
                                       'lgbm+tfidf',
                                       'xgb+tfidf'],
                                estimators_configs=[dict(n_estimators=20),
                                                    dict(n_estimators=20),
                                                    dict(n_estimators=20)])
builder.add_vectorizer_pipeline('bertVectorizer',
                                estimators=['svm',
                                            'etreesClf',
                                            'lgbmClf',
                                            'xgbClf'],
                                names=['svm+bert',
                                       'etrees+bert',
                                       'lgbm+bert',
                                       'xgb+bert'])

# Vamos aproveitar um conjunto de características
#   pré-extraídos se ele existir:
features = None
try:
    features = pd.read_csv('essay_br_extended_features.csv')
except Exception:
    ...

# Uma vez que tenhamos configurado o experimento,
#   podemos obter uma instância:
experiment = builder.build(features_df=features)

In [3]:
# === Executando o experimento ===
result = experiment.run()

[INFO] [aibox.nlp.experiments.simple_experiment] Setting up experiment...
[INFO] [aibox.nlp.experiments.simple_experiment] Obtaining train and test split...
[INFO] [aibox.nlp.experiments.simple_experiment] Train has 5251 samples, Test has 1313 samples.
[INFO] [aibox.nlp.experiments.simple_experiment] Run started.
[INFO] [aibox.nlp.experiments.simple_experiment] Started pipeline "xgb+features" (1/11)


Vetorização:   0%|          | 0/5251 [00:00<?, ?it/s]

Vetorização:   0%|          | 0/1313 [00:00<?, ?it/s]

[INFO] [aibox.nlp.experiments.simple_experiment] Started pipeline "lgbm+features" (2/11)


Vetorização:   0%|          | 0/5251 [00:00<?, ?it/s]

...

Vetorização:   0%|          | 0/1313 [00:00<?, ?it/s]

[INFO] [aibox.nlp.experiments.simple_experiment] Started pipeline "etrees+tfidf" (11/11)


Vetorização:   0%|          | 0/5251 [00:00<?, ?it/s]

Vetorização:   0%|          | 0/1313 [00:00<?, ?it/s]

[INFO] [aibox.nlp.experiments.simple_experiment] Run finished in 539.52 seconds.
Best pipeline: etrees+bert


In [4]:
# === Inspecionando os resultados ===
# Podemos obter o nome da melhor pipeline:
result.best_pipeline.name

'etrees+bert'

In [5]:
# Também podemos listar o valor das melhores métricas:
print(json.dumps({k: v.tolist() for k, v in result.best_metrics.items()},
                 indent=2,
                 ensure_ascii=False))

{
  "Weighted Precision": 0.6915916204452515,
  "Weighted Recall": 0.6755521893501282,
  "Weighted F1-score": 0.6246420741081238,
  "Kappa": 0.4168861210346222,
  "Neighbor Kappa": 0.4168861210346222
}


In [6]:
# Também podemos listar o valor de todas as métricas calculadas:
print(json.dumps({k: {k_: v_.tolist() for k_, v_ in v.items()} 
                  for k, v in result.metrics_history.items()},
                 indent=2,
                 ensure_ascii=False))

{
  "xgb+features": {
    "Weighted Precision": 0.4880306124687195,
    "Weighted Recall": 0.5300837755203247,
    "Weighted F1-score": 0.49591535329818726,
    "Kappa": 0.1701161414384842,
    "Neighbor Kappa": 0.1701161414384842
  },
  "lgbm+features": {
    "Weighted Precision": 0.4873538613319397,
    "Weighted Recall": 0.5277989506721497,
    "Weighted F1-score": 0.4935830533504486,
    "Kappa": 0.1650332659482956,
    "Neighbor Kappa": 0.1650332659482956
  },
  "etrees+features": {
    "Weighted Precision": 0.46003034710884094,
    "Weighted Recall": 0.5354150533676147,
    "Weighted F1-score": 0.48790010809898376,
    "Kappa": 0.16338065266609192,
    "Neighbor Kappa": 0.16338065266609192
  },
  "svm+features": {
    "Weighted Precision": 0.22876513004302979,
    "Weighted Recall": 0.4782939851284027,
    "Weighted F1-score": 0.3094988465309143,
    "Kappa": 0.0,
    "Neighbor Kappa": 0.0
  },
  "xgb+bert": {
    "Weighted Precision": 0.6563370823860168,
    "Weighted Recall": 0

In [7]:
# Por último, também é possível recuperar um DataFrame com as características
#   extraídas para os textos do dataset.
result.extras.df_features.head(3)

,text,adapted_dalechall,adverbs_before_main_verb_ratio,brunet_indice,clauses_per_sentence,coord_conj_ratio,coordinate_conjunctions_per_clauses,dialog_pron_ratio,easy_conj_ratio,flesch_indice,...,sentences_with_5_clauses,sentences_with_6_clauses,sentences_with_7_clauses,short_sentence_ratio,simple_word_ratio,std_noun_phrase,subord_conj_ratio,token_var_idx,verb_regency_score,words_before_main_verb
0,"Quando falamos em cultura cultura, logo pensam...",6.075747,1.000000,11.485317,4.8,0.545455,0.500000,0.083333,0.241803,32.183328,...,0,1,1,0.0,0.0,0.809721,0.454545,55.058231,0.833333,12
1,"Em uma época na qual vivemos hoje, onde o maio...",5.751029,0.333333,9.908926,6.0,0.454545,0.277778,0.000000,0.190476,39.019286,...,0,0,1,0.0,0.0,0.769217,0.545455,76.765148,1.000000,18
2,Atualmente a ciência já desmitificou vários fa...,6.830699,1.200000,10.952084,5.0,0.500000,0.450000,0.000000,0.266667,-10.617132,...,0,0,2,0.0,0.0,1.385570,0.500000,70.417395,1.000000,4


In [8]:
# Podemos salvar a extração de características 
#   para reutiliza rem outro momento.
from pathlib import Path

output = Path('essay_br_extended_features.csv')

if not output.exists():
    result.extras.df_features.to_csv(output,
                                     index=False)

In [9]:
# Também podemos realizar a serialização
#   da melhor pipeline
pipeline = result.best_pipeline
serialization.save_pipeline(pipeline, f'{pipeline.name}.joblib')

In [10]:
# Depois, podemos recuperar essa pipeline
#   e utilizá-la para novas predições
pipeline = serialization.load_pipeline(f'{pipeline.name}.joblib')
predictions = pipeline.predict(result.test_df.text.to_numpy())
assert (predictions == result.best_pipeline_test_predictions).all()

Vetorização:   0%|          | 0/1313 [00:00<?, ?it/s]

## Exemplo 2 - Utilizando Experimentos Pré-Inicializados

O pacote `aibox.nlp.factory` também oferece experimentos pré-inicializados para serem utilizados como baselines. Essencialmente, são utilizadas as configurações padrão para todos os estimadores, vetorizados e extratores de característica.


In [ ]:
# === Construindo um experimento para classificação no Essay-BR clássico ===

# Inicializando um experimento baseado em características com o uso
#   de estimadores clássicos (SVM, Random Forest, XGBoost, etc) e
#   métricas padrão para o problema (precision, recall, f1-score, kappa ou 
#   RMSE, MAE, R2 e MSE)
builder = SimpleExperimentBuilder.features_experiment(42, 'classification')

# Depois, só precisamos definir o dataset
builder.dataset('essayBR',
                extended=False,
                target_competence='C1')

# Vamos aproveitar um conjunto de características
#   pré-extraídos se ele existir:
features = None
try:
    features = pd.read_csv('essay_br_classic_features.csv')
except Exception:
    ...

# Uma vez que tenhamos configurado o experimento,
#   podemos obter uma instância:
experiment = builder.build(features_df=features)

In [ ]:
# === Executando o experimento ===
# A extração de todas as características
#   para o dataset pode demorar
#   consideravelmente (2h+)
result = experiment.run()

In [ ]:
# === Inspecionando os resultados ===
# Podemos obter o nome da melhor pipeline:
print('Melhor pipeline:', result.best_pipeline.name)

# Também podemos obter as métricas
print(json.dumps({k: v.tolist() for k, v in result.best_metrics.items()},
                 indent=2,
                 ensure_ascii=False))

In [ ]:
# Podemos salvar a extração de características
# para reutilizar em outro momento.
from pathlib import Path

output = Path('essay_br_classic_features.csv')

if not output.exists():
    result.extras.df_features.to_csv(output,
                                     index=False)